In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("/pokemons.csv")
pokemon_data = df.copy().drop(columns=['name', 'evolves_from', 'type1', 'type2', 'abilities', 'desc'])
pokemon_data

,id,rank,generation,hp,atk,def,spatk,spdef,speed,total,height,weight
0,1,ordinary,generation-i,45,49,49,65,65,45,318,7,69
1,2,ordinary,generation-i,60,62,63,80,80,60,405,10,130
2,3,ordinary,generation-i,80,82,83,100,100,80,525,20,1000
3,4,ordinary,generation-i,39,52,43,60,50,65,309,6,85
4,5,ordinary,generation-i,58,64,58,80,65,80,405,11,190
...,...,...,...,...,...,...,...,...,...,...,...,...
1012,1013,ordinary,generation-ix,71,60,106,121,80,70,508,2,22
1013,1014,legendary,generation-ix,88,128,115,58,86,80,555,18,922
1014,1015,legendary,generation-ix,88,75,66,130,90,106,555,10,122
1015,1016,legendary,generation-ix,88,91,82,70,125,99,555,14,301
